In [1]:
import pandas  as pd 
from itertools import product

Estado actual: 
* Pandas es muy lento para lo que quiero hacer
* Trabajo actual: separar los desempates en: Primero wins, despues rounds, despues mapas (probablemente no se implemente)
* Potenciales soluciones: Cambiar la manera de armar la tabla a Numpy (deberia ser lo mas rapido por mucho) Ventajas: velocidad y puedo evaluar por filas los resultados 

-------------------------------------------------------------------------

Arranco casi de 0, problemas con la unicacion de memoria de los objetos y se armo un re quilombo. 
Intento 2 diseño enfocado tabla final que tengo y estructurar desde ese punto hacia atras. 



In [2]:
#Esto es raro, siento que estoy dando vueltas en circulo con este diccionario creando los objetos y volviendo a usarlos 
#Lo termino usando con un zip, lo que puede ser peligroso con el orden y las keys
teamNameScores = {
                "name": ["Kru","Lev","100T","mibr","NRG","EG","FUR","LOUD", "2G","GAMERS2","C9","SEN"],
                "wins": [6,9,5,1,4,4,3,4,7,00,6,6],
                "loss":[4,1,5,9,6,6,7,6,3,00,4,4],
                "matchWins" : [15,19,12,3,10,10,8,11,15,00,13,13],
                "matchLoss" : [10,4,11,18,13,15,15,14,8,00,13,8],
                "roundWin":[287,296,255,168,238,241,195,249,249,00,270,249],
                "roundLoss":[247,204,232,257,237,281,270,275,214,00,276,214]
                
                  }

roundMatches = {
                "week11" : [["Kru","Lev"], ["100T","mibr"],["NRG","EG"], ["FUR","LOUD"],["2G","GAMERS2"],["C9","SEN"]],
                "week12" : [["Kru","100T"], ["Lev","mibr"],["NRG","FUR"], ["EG","LOUD"],["C9","GAMERS2"],["2G","SEN"]]
                }

In [13]:
teamNameScores = {
    "name": [
        "Kru",
        "Lev",
        "100T",
        "mibr",
        "NRG",
        "EG",
        "FUR",
        "LOUD",
        "2G",
        "GAMERS2",
        "C9",
        "SEN",
    ],
    "wins": [0, 0, 0, 0, 0, 0, 0, 0, 0, 00, 0, 0],
    "loss": [0, 0, 0, 0, 0, 0, 0, 0, 0, 00, 0, 0],
    "matchWins": [0, 0, 0,0, 0, 0,0, 0, 0, 0, 0, 0],
    "matchLoss": [0,0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0],
    "roundWin": [0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0],
    "roundLoss": [0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0],
}

roundMatches = {
    "week11": [
        ["Kru", "Lev"],
        ["100T", "mibr"],
        ["NRG", "EG"],
        ["FUR", "LOUD"],
        ["2G", "GAMERS2"],
        ["C9", "SEN"],
    ],
    "week12": [
        ["Kru", "100T"],
        ["Lev", "mibr"],
        ["NRG", "FUR"],
        ["EG", "LOUD"],
        ["C9", "GAMERS2"],
        ["2G", "SEN"],
    ],
}

In [36]:
from collections import defaultdict

In [41]:
def run_round(matches, teams, previous_results=None):
    if previous_results is None:
        previous_results = [] 
        for i, team in enumerate(teams["name"]):    
            temp_dict = {"UP" : 1,
            "Team": team,
            "wins": teams["wins"][i],
            "matchWins": teams["matchWins"][i],
            "matchLoss": teams["matchLoss"][i]
            }
            previous_results.append(temp_dict)

    prev_df = pd.DataFrame(previous_results)
    grouped_results = prev_df.groupby("UP")

    match_outcomes = list(product([0, 1, 2, 3], repeat=len(matches)))
    new_results = []

    result_mapping = {
        0: (0, 2),  # pierde 0-2
        1: (1, 2),  # pierde 1-2
        2: (2, 1),  # gana 2-1
        3: (2, 0),  # gana 2-0
    }

    
    test_group = defaultdict(list)
    for result in previous_results:
        test_group[result["UP"]].append(result)

    # Iterar sobre los grupos y construir simul_teams
    for up, group in test_group.items():
        simul_teams = {
            team_data["Team"]: {
                "wins": team_data["wins"],
                "matchWins": team_data["matchWins"],
                "matchLoss": team_data["matchLoss"],
            }
            for team_data in group}
            
    # for up, group in grouped_results:   
    #     simul_teams = {
    #         team_data["Team"]: {
    #             "wins": team_data["wins"],
    #             "matchWins": team_data["matchWins"],
    #             "matchLoss": team_data["matchLoss"],
    #         }
    #         for _, team_data in group.iterrows()
    #     }
    #     print(simul_teams)

        for combination_index, combination in enumerate(match_outcomes, start=1):

            temp_teams = {team: data.copy() for team, data in simul_teams.items()}

            for match_index, result in enumerate(combination):
                team1, team2 = matches[match_index]
                team1_score, team2_score = result_mapping[result]

                if result in [2, 3]:
                    temp_teams[team1]["wins"] += 1
                    temp_teams[team1]["matchWins"] += team1_score
                    temp_teams[team1]["matchLoss"] += team2_score
                    temp_teams[team2]["matchWins"] += team2_score
                    temp_teams[team2]["matchLoss"] += team1_score

                else:
                    temp_teams[team2]["wins"] += 1
                    temp_teams[team2]["matchWins"] += team2_score
                    temp_teams[team2]["matchLoss"] += team1_score
                    temp_teams[team1]["matchWins"] += team1_score
                    temp_teams[team1]["matchLoss"] += team2_score

            new_up = f"{up}-{combination_index}"

            for team, data in temp_teams.items():
                new_results.append(
                    {
                        "UP": new_up,
                        "Team": team,
                        "wins": data["wins"],
                        "matchWins": data["matchWins"],
                        "matchLoss": data["matchLoss"],
                    }
                )

    return new_results

In [15]:
teams = teamNameScores
matches = (("Kru","Lev"), ("100T","mibr"),("NRG","EG"), ("FUR","LOUD"),("2G","GAMERS2"),("C9","SEN"))
matches2 = [["Kru","100T"], ["Lev","mibr"],["NRG","FUR"], ["EG","LOUD"],["C9","GAMERS2"],["2G","SEN"]]


In [42]:
match1_results = run_round(matches, teams)

{'Kru': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'Lev': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, '100T': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'mibr': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'NRG': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'EG': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'FUR': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'LOUD': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, '2G': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'GAMERS2': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'C9': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'SEN': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}}
{'Kru': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'Lev': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, '100T': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'mibr': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'NRG': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'EG': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'FUR': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'LOUD

In [96]:
len(match1_results)/12

4096.0

In [21]:
match1_results

[{'UP': '1-1', 'Team': 'Kru', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'Lev', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': '100T', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'mibr', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': 'NRG', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'EG', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': 'FUR', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'LOUD', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': '2G', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'GAMERS2', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': 'C9', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'SEN', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-2', 'Team': 'Kru', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-2', 'Team': 'Lev', 'win

In [98]:
round1_df = pd.DataFrame(match1_results)

In [101]:
test_woking = round1_df

In [99]:
round1_df[round1_df["Team"] == "Lev"]

,UP,Team,wins,matchWins,matchLoss
1,1-1,Lev,1,2,0
13,1-2,Lev,1,2,0
25,1-3,Lev,1,2,0
37,1-4,Lev,1,2,0
49,1-5,Lev,1,2,0
...,...,...,...,...,...
49093,1-4092,Lev,0,0,2
49105,1-4093,Lev,0,0,2
49117,1-4094,Lev,0,0,2
49129,1-4095,Lev,0,0,2


In [115]:
round1_df.wins.value_counts()

0    24576
1    24576
Name: wins, dtype: int64

In [109]:
match2_results = run_round(matches, teams, match1_results)

In [117]:
round2_df = pd.DataFrame(match2_results)



In [124]:
round2_df[500000:600000].head()

,UP,Team,wins,matchWins,matchLoss
500000,1-1007-707,2G,1,2,2
500001,1-1007-707,GAMERS2,1,2,2
500002,1-1007-707,C9,2,4,2
500003,1-1007-707,SEN,0,2,4
500004,1-1007-708,Kru,0,0,4


In [135]:
def run_round2(matches, teams, previous_results=None):
    if previous_results is None:
        previous_results = [
            {
                "UP": 1,
                "Team": team,
                "wins": data["wins"],
                "matchWins": data["matchWins"],
                "matchLoss": data["matchLoss"],
            }
            for team, data in teams.items()
        ]

    match_outcomes = list(product([0, 1, 2, 3], repeat=len(matches)))
    new_results = []

    result_mapping = {
        0: (0, 2),  # pierde 0-2
        1: (1, 2),  # pierde 1-2
        2: (2, 1),  # gana 2-1
        3: (2, 0),  # gana 2-0
    }
    setUP = set()
    for result in previous_results:
        setUP.add(result["UP"])
    #print(setUP)
    
   
    
    for up in setUP:        
        simul_teams = {}
        for result in previous_results:
   
            simul_teams[result["Team"]] = {
                "wins": result["wins"],
                "matchWins": result["matchWins"],
                "matchLoss": result["matchLoss"],
            }
            
        for combination_index, combination in enumerate(match_outcomes, start=1):
            temp_teams = {team: data.copy() for team, data in simul_teams.items()}

            for match_index, result in enumerate(combination):
                team1, team2 = matches[match_index]
                team1_score, team2_score = result_mapping[result]

                #print(result)
                if result in [2, 3]:
                    temp_teams[team1]["wins"] += 1
                    temp_teams[team1]["matchWins"] += team1_score
                    temp_teams[team1]["matchLoss"] += team2_score
                    temp_teams[team2]["matchWins"] += team2_score
                    temp_teams[team2]["matchLoss"] += team1_score
                    # print(team1, team2, match_wins, match_loss)

                else:
                    temp_teams[team2]["wins"] += 1
                    temp_teams[team2]["matchWins"] += team2_score
                    temp_teams[team2]["matchLoss"] += team1_score
                    temp_teams[team1]["matchWins"] += team1_score
                    temp_teams[team1]["matchLoss"] += team2_score

            new_up = f"{up}-{combination_index}"

            for team, data in temp_teams.items():
                new_results.append(
                    {
                        "UP": new_up,
                        "Team": team,
                        "wins": data["wins"],
                        "matchWins": data["matchWins"],
                        "matchLoss": data["matchLoss"],
                    }
                )            
    return new_results

In [128]:
teams = hierarchy_dict
matches = (("Kru","Lev"), ("100T","mibr"),("NRG","EG"), ("FUR","LOUD"),("2G","GAMERS2"),("C9","SEN"))

In [136]:
match1_results = run_round2(matches, teams)

{'Kru': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'Lev': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, '100T': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'mibr': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'NRG': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'EG': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'FUR': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'LOUD': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, '2G': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'GAMERS2': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'C9': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}, 'SEN': {'wins': 0, 'matchWins': 0, 'matchLoss': 0}}


In [130]:
match1_results

[{'UP': '1-1', 'Team': 'Kru', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'Lev', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': '100T', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'mibr', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': 'NRG', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'EG', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': 'FUR', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'LOUD', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': '2G', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'GAMERS2', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-1', 'Team': 'C9', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-1', 'Team': 'SEN', 'wins': 1, 'matchWins': 2, 'matchLoss': 0},
 {'UP': '1-2', 'Team': 'Kru', 'wins': 0, 'matchWins': 0, 'matchLoss': 2},
 {'UP': '1-2', 'Team': 'Lev', 'win

In [ ]:
match2_results = run_round2(matches, teams,match1_results)

{'Kru': {'wins': 1, 'matchWins': 2, 'matchLoss': 0}, 'Lev': {'wins': 0, 'matchWins': 0, 'matchLoss': 2}, '100T': {'wins': 1, 'matchWins': 2, 'matchLoss': 0}, 'mibr': {'wins': 0, 'matchWins': 0, 'matchLoss': 2}, 'NRG': {'wins': 1, 'matchWins': 2, 'matchLoss': 0}, 'EG': {'wins': 0, 'matchWins': 0, 'matchLoss': 2}, 'FUR': {'wins': 1, 'matchWins': 2, 'matchLoss': 0}, 'LOUD': {'wins': 0, 'matchWins': 0, 'matchLoss': 2}, '2G': {'wins': 1, 'matchWins': 2, 'matchLoss': 0}, 'GAMERS2': {'wins': 0, 'matchWins': 0, 'matchLoss': 2}, 'C9': {'wins': 1, 'matchWins': 2, 'matchLoss': 0}, 'SEN': {'wins': 0, 'matchWins': 0, 'matchLoss': 2}}


In [134]:
len(match2_results)

201326592